In [1]:
# 2nd run of trial
# removed number of mosquitoes since test set does not have and it confuses the model

In [2]:
import pandas as pd
import numpy as np
train = pd.read_csv('../assets/train_2.csv')

In [3]:
train.shape

(4616, 193)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier

In [5]:
pd.options.display.max_columns = None
train.head()

,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,NumMosquitos,WnvPresent,Trap_T002,Trap_T002A,Trap_T002B,Trap_T003,Trap_T004,Trap_T005,Trap_T006,Trap_T007,Trap_T008,Trap_T009,Trap_T011,Trap_T012,Trap_T013,Trap_T014,Trap_T015,Trap_T016,Trap_T017,Trap_T018,Trap_T019,Trap_T025,Trap_T027,Trap_T028,Trap_T030,Trap_T031,Trap_T033,Trap_T034,Trap_T035,Trap_T036,Trap_T037,Trap_T039,Trap_T040,Trap_T043,Trap_T044,Trap_T045,Trap_T046,Trap_T047,Trap_T048,Trap_T049,Trap_T050,Trap_T051,Trap_T054,Trap_T054C,Trap_T060,Trap_T061,Trap_T062,Trap_T063,Trap_T065,Trap_T065A,Trap_T066,Trap_T067,Trap_T069,Trap_T070,Trap_T071,Trap_T072,Trap_T073,Trap_T074,Trap_T075,Trap_T076,Trap_T077,Trap_T078,Trap_T079,Trap_T080,Trap_T081,Trap_T082,Trap_T083,Trap_T084,Trap_T085,Trap_T086,Trap_T088,Trap_T089,Trap_T090,Trap_T090A,Trap_T090B,Trap_T090C,Trap_T091,Trap_T092,Trap_T094,Trap_T094B,Trap_T095,Trap_T096,Trap_T097,Trap_T099,Trap_T100,Trap_T102,Trap_T103,Trap_T107,Trap_T114,Trap_T115,Trap_T128,Trap_T128A,Trap_T129,Trap_T135,Trap_T138,Trap_T141,Trap_T142,Trap_T143,Trap_T144,Trap_T145,Trap_T146,Trap_T147,Trap_T148,Trap_T149,Trap_T150,Trap_T151,Trap_T152,Trap_T153,Trap_T154,Trap_T155,Trap_T156,Trap_T157,Trap_T158,Trap_T159,Trap_T160,Trap_T161,Trap_T162,Trap_T200,Trap_T200A,Trap_T200B,Trap_T206,Trap_T209,Trap_T212,Trap_T215,Trap_T218,Trap_T218A,Trap_T218B,Trap_T218C,Trap_T219,Trap_T220,Trap_T221,Trap_T222,Trap_T223,Trap_T224,Trap_T225,Trap_T226,Trap_T227,Trap_T228,Trap_T229,Trap_T230,Trap_T231,Trap_T232,Trap_T233,Trap_T234,Trap_T235,Trap_T236,Trap_T237,Trap_T238,Trap_T900,Trap_T903,Month,Day,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,SQ,SN,VCFG,GR
0,2007-05-29,41.954690,-87.800991,0,1,1,0,0,0,0,2.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,29,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,0 days 04:21:00.000000000,0 days 19:17:00.000000000,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2007-05-29,41.994991,-87.769279,0,0,1,0,0,0,0,1.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,29,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,0 days 04:21:00.000000000,0 days 19:17:00.000000000,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2007-05-29,41.974089,-87.824812,0,1,1,0,0,0,0,5.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,29,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,0 days 04:21:00.000000000,0 days 19:17:00.000000000,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2007-05-29,41.921600,-87.666455,0,0,1,0,0,0,0,2.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,29,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,0 days 04:21:00.000000000,0 days 19:17:00.000000000,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2007-05-29,41.891118,-87.654491,0,0,1,0,0,0,0,1.0,0.0,0

In [6]:
train['Sunrise'] = pd.to_timedelta(train['Sunrise'])
train['Sunset'] = pd.to_timedelta(train['Sunset'])

### Selecting columns to include to X

In [7]:
X_1 = pd.concat([train.iloc[:,1:3],train.iloc[:,160:170]],axis = 1)
X_prescale = pd.concat([X_1,train.iloc[:,172:178]],axis = 1)
ss = StandardScaler()
Xs = pd.DataFrame(ss.fit_transform(X_prescale), columns=X_prescale.columns)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [8]:
X = pd.concat([train.iloc[:,3:10],train.iloc[:,178:193],Xs],axis = 1)
y = train.iloc[:,11]

In [9]:
X.shape

(4616, 40)

In [10]:
X.head(2)

,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,SQ,SN,VCFG,GR,Latitude,Longitude,Month,Day,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.018627,-1.316134,-2.326452,1.505091,0.819344,-0.016868,0.447821,1.146334,-0.053207,0.221695,-0.377749,0.40859,-0.461259,1.108795,1.112219,-0.089381,-0.088263,-0.243311
1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.402268,-0.925219,-2.326452,1.505091,0.819344,-0.016868,0.447821,1.146334,-0.053207,0.221695,-0.377749,0.40859,-0.461259,1.108795,1.112219,-0.089381,-0.088263,-0.243311


In [11]:
#Train,test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20 , train_size=0.7 , test_size=0.3, stratify=y)

### Using a loop to run through different models, and using gridsearch to find optimial parameters
### we wish to find the best model to predict the most accurate scores

In [12]:
model = []
parameters = []
scores = []
precision = []
recall = []
ra_score = []
pred = []

# To put the name of model, models from sklearn, and hyperparameters of the model we want to search through in order

names = ['Decision Tree', 'RandomForest', 'LogReg','KNN','SVC','AdaBoost', 'BaggingClass', 'GradientBoost']

estimators = [DecisionTreeClassifier(), RandomForestClassifier(), LogisticRegression(), KNeighborsClassifier(),
             SVC(), AdaBoostClassifier(), BaggingClassifier(), GradientBoostingClassifier()]

params = [
    {'clf__max_depth': [None, 10, 20, 30, 50],
    'clf__min_samples_split': [5,10,15,20,25,30]}, #DecisionTree

    {'clf__max_depth': [None,50,70,100,150],
     'clf__min_samples_split': [5,10,15,20,30]}, #RandomForest
    
    {'clf__C':[1,3,5,10],
     'clf__solver':['lbfgs', 'liblinear', 'sag', 'saga'],
     'clf__max_iter':[150,200,500]}, #LogisticRegression
    
    {'clf__n_neighbors':[3,5,10,15],
     'clf__weights':['uniform','distance'],
     'clf__p':[1,2]}, #KNeighborsClassifier
    
    {'clf__kernel':['rbf','linear']}, #SVC - 'something wrong with 'sigmoid','poly'
    
    {'clf__n_estimators':[50,100,200,500]}, #AdaBoost
    
    {'clf__n_estimators':[10,30,50,100],
     'clf__max_samples':[1,5,10],
     'clf__max_features':[1,5,10]}, #BaggingClassifier
    
    {'clf__loss' : ['deviance', 'exponential'],
     'clf__n_estimators':[50,100,200,500],
     'clf__max_features': [None, 'auto','sqrt','log2']} #GradientBoosting
]

for name, estimator, param in zip(names, estimators, params):
    clf_pipe = Pipeline([
        # not sure if we need to scale the X
        # removed ('ss', StandardScaler),
        ('clf', estimator)
    ])
#     cv = StratifiedKFold(shuffle=True, n_splits=3)
    cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
    gs = GridSearchCV(clf_pipe, param_grid=param, cv=cv, verbose=10, n_jobs=-1)
    gs.fit(X_train, y_train)
    
    print(name)
    print('Best Score: {}'.format(gs.best_score_))
    print('Best Params: {}'.format(gs.best_params_))
    
    # Best Model
    grid_model = gs.best_estimator_
    
    print('Model Score on X_train: {}'.format(grid_model.score(X_train, y_train)))
    
    # Prediction and score
    pred = grid_model.predict(X_test)
    print('Accuracy score of prediction: {}'.format(accuracy_score(y_test, pred)))
        
    # Confusion Matrix 
    conf_matrix = confusion_matrix(y_test, pred)
    
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    print('TN: {}, FP:{}, FN:{}, TP:{}'.format(tn,fp,fn,tp))
    print('Precision of the model is {}'. format(tp/(tp+fp)))
    print('Recall of the model is {}'.format(tp/(tp+fn)))
    
    #roc_auc
    ra = roc_auc_score(y_test, pred)
    print ('ROC_AUC: {}'.format(ra))
    
    print('\n')
    model.append(name)
    parameters.append(gs.best_params_)
    scores.append(accuracy_score(y_test, pred))
    recall.append(tp/(tp+fn))
    precision.append(tp/(tp+fp))
    ra_score.append(ra)
    
results = pd.DataFrame({'model': model, 
              'parameters': parameters, 
              'scores': scores, 
              'recall': recall, 
              'precision': precision,
              'roc_auc':ra_score
                       })
results

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1959s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  67 out of  90 | elapsed:    4.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  77 out of  90 | elapsed:    4.5s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  87 out of  90 | elapsed:    4.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Decision Tree
Best Score: 0.9108706852138073
Best Params: {'clf__max_depth': 10, 'clf__min_samples_split': 30}
Model Score on X_train: 0.9362426493345714
Accuracy score of prediction: 0.9068592057761733
TN: 1233, FP:36, FN:93, TP:23
Precision of the model is 0.3898305084745763
Recall of the model is 0.19827586206896552
ROC_AUC: 0.5849535338713622


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1228s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  36 out of  75 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  44 out of  75 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  52 out of  75 | elapsed:    0.8s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  60 out of  75 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  68 out of  75 | elapsed:    0.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    1.0s finished
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20

RandomForest
Best Score: 0.9191138588356518
Best Params: {'clf__max_depth': 50, 'clf__min_samples_split': 20}
Model Score on X_train: 0.9316001238006809
Accuracy score of prediction: 0.916245487364621
TN: 1256, FP:13, FN:103, TP:13
Precision of the model is 0.5
Recall of the model is 0.11206896551724138
ROC_AUC: 0.550912339338605


Fitting 3 folds for each of 48 candidates, totalling 144 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0472s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  19 out of 108 | elapsed:    0.4s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    2.7s finished
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in long_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in long_scalars
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


BaggingClass
Best Score: 0.9180834621329211
Best Params: {'clf__max_features': 5, 'clf__max_samples': 10, 'clf__n_estimators': 10}
Model Score on X_train: 0.9167440420922315
Accuracy score of prediction: 0.916245487364621
TN: 1269, FP:0, FN:116, TP:0
Precision of the model is nan
Recall of the model is 0.0
ROC_AUC: 0.5


Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  83 out of  96 | elapsed:   15.4s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  93 out of  96 | elapsed:   17.2s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:   17.6s finished


GradientBoost
Best Score: 0.9237506439979392
Best Params: {'clf__loss': 'deviance', 'clf__max_features': None, 'clf__n_estimators': 50}
Model Score on X_train: 0.9275766016713092
Accuracy score of prediction: 0.9184115523465703
TN: 1265, FP:4, FN:109, TP:7
Precision of the model is 0.6363636363636364
Recall of the model is 0.0603448275862069
ROC_AUC: 0.5285963696638678




,model,parameters,scores,recall,precision,roc_auc
0,Decision Tree,"{'clf__max_depth': 10, 'clf__min_samples_split...",0.906859,0.198276,0.389831,0.584954
1,RandomForest,"{'clf__max_depth': 50, 'clf__min_samples_split...",0.916245,0.112069,0.500000,0.550912
2,LogReg,"{'clf__C': 3, 'clf__max_iter': 150, 'clf__solv...",0.915523,0.017241,0.400000,0.507439
3,KNN,"{'clf__n_neighbors': 10, 'clf__p': 1, 'clf__we...",0.915523,0.086207,0.476190,0.538769
4,SVC,{'clf__kernel': 'rbf'},0.916245,0.000000,NaN,0.500000
5,AdaBoost,{'clf__n_estimators': 200},0.920578,0.172414,0.588235,0.580691
6,BaggingClass,"{'clf__max_features': 5, 'clf__max_samples': 1...",0.916245,0.000000,NaN,0.500000
7,GradientBoost,"{'clf__loss': 'deviance', 'clf__max_features':...",0.918412,0.060345,0.636364,0.528596


### Using the below as our best model

In [13]:
AdaBoostClassifier()

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [14]:
model = AdaBoostClassifier(n_estimators=50)
ada = model.fit(X_train, y_train)

In [15]:
pred = grid_model.predict_proba(X_test)[:,1]

In [16]:
pred

array([0.02681693, 0.01767025, 0.00600223, ..., 0.04869968, 0.08059326,
       0.01192135])

In [17]:
roc_auc_score(y_test, pred)

0.8538151137197361

### Preprocessing the same way for the test set

In [18]:
test = pd.read_csv('../assets/test.csv')

In [19]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [20]:
test.shape

(116293, 11)

In [21]:
test = test.drop(['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy','Trap'],axis=1)

In [22]:
column = ['Species']

test = pd.get_dummies(data = test, prefix=column, columns = column, drop_first=True)

In [23]:
weather = pd.read_csv('../assets/weather_2.csv')

In [24]:
test_2 = pd.merge(test,weather, on='Date')

In [25]:
test_2.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,Year,Month,Day,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,SQ,SN,VCFG,GR
0,1,2008-06-11,41.95469,-87.800991,0,1,0,0,0,0,0,2008,6,11,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0 days 04:16:00.000000000,0 days 19:26:00.000000000,0.0,29.31,29.98,9.15,18.0,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2008-06-11,41.95469,-87.800991,0,0,1,0,0,0,0,2008,6,11,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0 days 04:16:00.000000000,0 days 19:26:00.000000000,0.0,29.31,29.98,9.15,18.0,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2008-06-11,41.95469,-87.800991,1,0,0,0,0,0,0,2008,6,11,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0 days 04:16:00.000000000,0 days 19:26:00.000000000,0.0,29.31,29.98,9.15,18.0,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2008-06-11,41.95469,-87.800991,0,0,0,1,0,0,0,2008,6,11,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0 days 04:16:00.000000000,0 days 19:26:00.000000000,0.0,29.31,29.98,9.15,18.0,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2008-06-11,41.95469,-87.800991,0,0,0,0,0,1,0,2008,6,11,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0 days 04:16:00.000000000,0 days 19:26:00.000000000,0.0,29.31,29.98,9.15,18.0,10.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
test_2.iloc[:,30:45].head(2)

,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,SQ,SN,VCFG,GR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
X_1 = pd.concat([test_2.iloc[:,2:4],test_2.iloc[:,12:22]],axis = 1)
X_prescale = pd.concat([X_1,test_2.iloc[:,24:30]],axis = 1)
Xs = pd.DataFrame(ss.fit_transform(X_prescale), columns=X_prescale.columns)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [28]:
X = pd.concat([test_2.iloc[:,4:11],test_2.iloc[:,30:45],Xs],axis = 1)

In [29]:
final_pred = grid_model.predict_proba(X)[:,1]

In [30]:
final_pred

array([0.0152121 , 0.00861176, 0.02442091, ..., 0.01182267, 0.01182267,
       0.01182267])

### From the final prediction, combine to create a new database for submission

In [31]:
sub = pd.DataFrame()

In [32]:
sample = pd.read_csv('../assets/samplesubmission.csv')

In [33]:
sub['Id']=sample['Id']

In [34]:
sub['WnvPresent'] = final_pred

In [35]:
sub.shape

(116293, 2)

In [36]:
sub.head()

,Id,WnvPresent
0,1,0.015212
1,2,0.008612
2,3,0.024421
3,4,0.008612
4,5,0.008612


In [37]:
sub.to_csv('submission/sub3.csv',index=False)